In [1]:
import scrapfly_scrapers.linkedin as linkedin

# enable scrapfly cache
linkedin.BASE_CONFIG["cache"] = True
linkedin.BASE_CONFIG["debug"] = True


In [2]:

print("running Linkedin scrape and saving results to ./results directory")

# profile_data = await linkedin.scrape_profile(
#     urls=[
#         "https://www.linkedin.com/in/williamhgates"
#     ]
# )

from pprint import pprint
# pprint(profile_data)


# with open(output.joinpath("profile.json"), "w", encoding="utf-8") as file:
#     json.dump(profile_data, file, indent=2, ensure_ascii=False)    

# company_data = await linkedin.scrape_company(
#     urls=[
#         "https://linkedin.com/company/microsoft",
#         "https://linkedin.com/company/google",
#         "https://linkedin.com/company/apple"
#     ]
# )
# with open(output.joinpath("company.json"), "w", encoding="utf-8") as file:
#     json.dump(company_data, file, indent=2, ensure_ascii=False)    

# job_search_data = await linkedin.scrape_job_search(
#     # it include other search parameters, refer to the search pages on browser for more details
#     keyword="Python Developer",
#     location="United States",
#     max_pages=3
# )
# with open(output.joinpath("job_search.json"), "w", encoding="utf-8") as file:
#     json.dump(job_search_data, file, indent=2, ensure_ascii=False)    

# job_data = await linkedin.scrape_jobs(
#     urls=[
#         "https://www.linkedin.com/jobs/view/data-center-engineering-operations-engineer-hyd-infinity-dceo-at-amazon-web-services-aws-4017265505",
#         "https://www.linkedin.com/jobs/view/content-strategist-google-cloud-content-strategy-and-experience-at-google-4015776107",
#         "https://www.linkedin.com/jobs/view/sr-content-marketing-manager-brand-protection-brand-protection-at-amazon-4007942181"
#     ]
# )
# with open(output.joinpath("jobs.json"), "w", encoding="utf-8") as file:
#     json.dump(job_data, file, indent=2, ensure_ascii=False)

running Linkedin scrape and saving results to ./results directory


In [24]:
from google_search import search, SearchResult

linkedin_pages = list(search("site:www.linkedin.com/company/ intitle:98point6"))
linkedin_pages

2024-09-27 13:46:51.378 | DEBUG    | google_search:search:58 - Google search results: {'kind': 'customsearch#search', 'url': {'type': 'application/json', 'template': 'https://www.googleapis.com/customsearch/v1?q={searchTerms}&num={count?}&start={startIndex?}&lr={language?}&safe={safe?}&cx={cx?}&sort={sort?}&filter={filter?}&gl={gl?}&cr={cr?}&googlehost={googleHost?}&c2coff={disableCnTwTranslation?}&hq={hq?}&hl={hl?}&siteSearch={siteSearch?}&siteSearchFilter={siteSearchFilter?}&exactTerms={exactTerms?}&excludeTerms={excludeTerms?}&linkSite={linkSite?}&orTerms={orTerms?}&dateRestrict={dateRestrict?}&lowRange={lowRange?}&highRange={highRange?}&searchType={searchType}&fileType={fileType?}&rights={rights?}&imgSize={imgSize?}&imgType={imgType?}&imgColorType={imgColorType?}&imgDominantColor={imgDominantColor?}&alt=json'}, 'queries': {'request': [{'title': 'Google Custom Search - site:www.linkedin.com/company/ intitle:98point6', 'totalResults': '2', 'searchTerms': 'site:www.linkedin.com/compan

[SearchResult(title='98point6 Technologies Inc. | LinkedIn', link='https://www.linkedin.com/company/98point6-tech-inc', snippet='98point6 Technologies Inc. | 10207 followers on LinkedIn. Expand your virtual clinic with technology designed by clinicians for clinicians.', formattedUrl='https://www.linkedin.com/company/98point6-tech-inc'),
 SearchResult(title='98POINT6 PHYSICIANS PC | LinkedIn', link='https://www.linkedin.com/company/98point6-physicians-pc', snippet='98POINT6 PHYSICIANS PC | 6 followers on LinkedIn.', formattedUrl='https://www.linkedin.com/company/98point6-physicians-pc')]

In [5]:
# Took 26 seconds?!?

company_data = await linkedin.scrape_company(
    urls=[
        linkedin_pages[0].link
    ]
)

pprint(company_data)

2024-09-27 13:02:07.521 | SUCCESS  | scrapfly_scrapers.linkedin:scrape_company:148 - scraped 1 companies from Linkedin


[{'life': {'affiliatedPages': [],
           'leaders': [],
           'similarPages': [{'address': 'San Francisco, CA',
                             'industry': 'Hospitals and Health Care',
                             'linkeinUrl': 'https://www.linkedin.com/company/transcarent',
                             'name': 'Transcarent'},
                            {'address': 'San Francisco, California',
                             'industry': 'Software Development',
                             'linkeinUrl': 'https://www.linkedin.com/company/hinge-health',
                             'name': 'Hinge Health'},
                            {'address': 'Portland, OR',
                             'industry': 'IT Services and IT Consulting',
                             'linkeinUrl': 'https://www.linkedin.com/company/bright-md',
                             'name': 'Bright.md'},
                            {'address': 'Seattle, Washington',
                             'industry': 'Hospitals 

In [10]:
# try scraping manually
from scrapfly import ScrapeConfig, ScrapflyClient
import os
scrape_config = ScrapeConfig(linkedin_pages[0].link, **linkedin.BASE_CONFIG)
scrape_config

client = ScrapflyClient(key=os.environ["SCRAPFLY_KEY"])
response = client.scrape(scrape_config)
response

In [12]:
response.response.text

'��result�\x00\x18�format�text�screenshots��response_headers�\x00\x13�content-length�27589�content-encoding�gzip�vary�Accept-Encoding�x-frame-options�sameorigin�x-li-proto�http/2�content-type�text/html; charset=utf-8�set-cookie�\x02�JSESSIONID=ajax:7182067584837755444; SameSite=None; Path=/; Domain=.www.linkedin.com; Secure, lang=v=2&lang=en-us; SameSite=None; Path=/; Domain=linkedin.com; Secure, bcookie="v=2&5e5bb9e1-9964-48b8-8991-668f4f7e94b0"; domain=.linkedin.com; Path=/; Secure; Expires=Sat, 27-Sep-2025 18:57:47 GMT; SameSite=None, bscookie="v=1&202409271857476bd261b1-fdb3-4011-8c36-0b2e21e5306fAQFdEdD7Phy2tGB6sC5xVIxWtFhNE3sT"; domain=.www.linkedin.com; Path=/; Secure; Expires=Sat, 27-Sep-2025 18:57:47 GMT; HttpOnly; SameSite=None, lidc="b=VGST07:s=V:r=V:a=V:p=V:g=3066:u=1:x=1:i=1727463467:t=1727549867:v=2:sig=AQFdwYe5sO0_Qaz2RdEr6GHyfdrzjvUZ"; Expires=Sat, 28 Sep 2024 18:57:47 GMT; domain=.linkedin.com; Path=/; SameSite=None; Secure�strict-transport-security�max-age=31536000�x-

In [14]:
import json

for script in json.loads(response.selector.xpath("//script[@type='application/ld+json']/text()").get()):
    print(script)

@context
@type
name
url
address
description
numberOfEmployees
logo
slogan
sameAs


In [15]:
response.selector.xpath("//script[@type='application/ld+json']/text()").get()

'\n      {"@context":"http://schema.org","@type":"Organization","name":"98point6 Technologies Inc.","url":"https://www.linkedin.com/company/98point6-tech-inc","address":{"type":"PostalAddress","streetAddress":"701 5th Avenue","addressLocality":"Seattle","addressRegion":"Washington","postalCode":"98164","addressCountry":"US"},"description":"98point6 Technologies builds and licenses software to power exceptional virtual care experiences for providers and patients based on years of clinical use across millions of patients. Its flagship 98point6 Technology Platform consists of turnkey, cloud-based software that harnesses the power of artificial intelligence bundled with time-tested, automated practice standards. The platform includes a physician-facing Clinician Console, patient-facing branded app and professional services and support. Available as licensed software, the 98point6 Technology Platform empowers health systems to create and grow premium virtual care offerings that give provide

In [16]:
for element in response.selector.xpath("//div[contains(@data-test-id, 'about-us')]"):
    print(element)

<div class="mb-2 flex papabear:mr-3 mamabear:mr-3 babybear:flex-wrap" data-test-id="about-us__website">
      <dt class="font-sans text-md font-bold text-color-text break-words w-40 papabear:flex-shrink-0 papabear:mr-3 mamabear:flex-shrink-0 mamabear:mr-3 babybear:mb-1">
          Website
      </dt>
      <dd class="font-sans text-md text-color-text break-words overflow-hidden">
        
                  
    <a class="link-no-visited-state hover:no-underline" data-tracking-control-name="about_website" href="https://www.linkedin.com/redir/redirect?url=https%3A%2F%2Fwww%2E98point6%2Ecom%2F&amp;urlhash=hmfb&amp;trk=about_website" target="_blank" rel="noopener" data-tracking-will-navigate aria-describedby="websiteLinkDescription">
      https://www.98point6.com/
      <icon class="align-middle" data-delayed-url="https://static.licdn.com/aero-v1/sc/h/49229g4q0jsla5l1xojq1wob9" aria-hidden="true"></icon>
    </a>
  
                  <p class="hidden" id="websiteLinkDescription">External 

In [18]:


for element in response.selector.xpath('//*[@id="fie-impression-container"]'):
    print(element)